THINGS TO DO (IN ORDER)
- import everything
- get the data
- analyzze data
  - oversample if needed
  - clean up anything weird (there shouldn't be much)
- split the data (train and test)
- training data
  - find best parameters for this / different models
- test the data
- get all results
- map results
- train CNN
- train Transformer


In [1]:
!git clone https://github.com/Samintha-C/CSE144-G9.git
%cd CSE144-G9/

Cloning into 'CSE144-G9'...
remote: Enumerating objects: 1306, done.
remote: Counting objects: 100% (1306/1306), done.
remote: Compressing objects: 100% (1300/1300), done.
remote: Total 1306 (delta 16), reused 1265 (delta 3), pack-reused 0
Receiving objects: 100% (1306/1306), 45.66 MiB | 14.40 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/CSE144-G9


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format="retina"
import numpy as np
import random
import torch
from torch import nn, optim
import math
from IPython import display
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
import pdb
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn import metrics
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
from torchvision import transforms, models
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torch.utils.data import Dataset
from PIL import Image, ImageEnhance


In [3]:
#traindf = pd.read_csv("_classes.csv")
#traindf.describe()
#traindf[""]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# Data loader function to load data into batches

class Data(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        csv_file: Path to the csv file.
        root_dir: Directory with all the images.
        transform: transform to be applied on a sample. optional
        """
        self.frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.frame.iloc[idx, 0])
        image = Image.open(img_name)
        #labels = self.frame.iloc[idx, 1:].to_numpy()
        # Convert labels to single class labels (assuming one-hot encoding)
        #labels = np.argmax(labels, axis=0)  # Find index of the class with highest probability
        #labels = torch.tensor(labels).long()  # Convert to LongTensor for CrossEntropyLoss
        labels = self.frame.iloc[idx, 1:].to_numpy()
        labels = labels.astype(np.float32)  # Convert labels to float32
        labels = torch.tensor(labels).float()  # Convert to FloatTensor for BCEWithLogitsLoss
        if self.transform:
            image = self.transform(image)

        return image, labels

transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
])

# Dataset and DataLoader
train_dataset = Data(csv_file='Dataset/train/_classes.csv', root_dir='Dataset/train/', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

validation_dataset = Data(csv_file='Dataset/valid/_classes.csv', root_dir='Dataset/valid/', transform=transform)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=32, shuffle=False)


test_dataset = Data('Dataset/test/_classes.csv', root_dir='Dataset/test/', transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


In [5]:
num_classes = 6
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Output: (32, 416, 416)
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),       # Output: (32, 208, 208)
            nn.Dropout(0.25))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),            # Output: (64, 206, 206)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),                             # Output: (64, 103, 103)
            nn.Dropout(0.25))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3),           # Output: (128, 101, 101)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),                             # Output: (128, 50, 50)
            nn.Dropout(0.25))

        self.fc1 = nn.Linear(128 * 50 * 50, 500)         # Adjusted input size
        self.fc2 = nn.Linear(500, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class OptimizedCNN(nn.Module):
    def __init__(self, num_classes=6):
        super(OptimizedCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.conv6 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(1024)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.dropout = nn.Dropout(p=0.5)

        self.fc1 = nn.Linear(1024 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 1024)
        self.fc3 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))
        x = self.pool(F.relu(self.bn6(self.conv6(x))))

        x = x.view(-1, 1024 * 6 * 6)

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)

        return x

model = OptimizedCNN(num_classes=6)


In [7]:
'''EPOCHS = 3

def train():
    model.train()
    for epoch in range(EPOCHS):
        #Note: 242 batches in epoch
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            if i % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, i * len(data), len(train_loader.dataset),
                    100. * i / len(train_loader), loss.item()))


    print('Done Training')
train()'''
x=1

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets

class CNNModel(nn.Module):
    def __init__(self, num_classes=6):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(512 * 13 * 13, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(-1, 512 * 13 * 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x

# Create a model instance
model = CNNModel(num_classes=6)

# Print model architecture
print(model)

# Define a loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Example usage with dummy data
# Assuming `train_loader` is a DataLoader object that provides the training data
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop (simplified example)
def train_model(model, criterion, optimizer, train_loader, num_epochs=10):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i in train_loader:
            # Zero the parameter gradients
            optimizer.zero_grad()
            images = i[0]
            # Concatenate labels into a single tensor
            labels = torch.cat(i[1:], dim=1)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Example transform for the dataset
transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
])

# Example dataset and dataloader
# Assuming `data_dir` is the directory with your dataset

# Train the model (this will need actual data in the 'data/train' directory)
train_model(model, criterion, optimizer, train_loader, num_epochs=10)

CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=86528, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=6, bias=True)
)


In [ ]:
import torch.optim as optim
from IPython import display

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def evaluate_model(model, dataloader, is_test=False):
    # Evaluation
    model.eval()

    with torch.no_grad():
        total_correct = 0
        total_loss = 0

        criterion = torch.nn.CrossEntropyLoss(reduction='none')

        for data, targets in dataloader:
            data = data.to(device)
            targets = [target.to(device) for target in targets]

            outputs = model(data)
            losses = [torch.sum(criterion(output, target)).item() for output, target in zip(outputs, targets)]
            total_loss += sum(losses)

            correct = 0
            for output, target in zip(outputs, targets):
                pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
                correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
            total_correct += correct

        total_loss /= len(dataloader.dataset)
        accuracy = 100. * total_correct / (len(dataloader.dataset) * len(targets))
        print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            "Test" if is_test else "Validation",
            total_loss, total_correct, len(dataloader.dataset) * len(targets),
            accuracy))

    # Set model back to training mode
    model.train()

def train_model(model, train_loader, optimizer, epochs=30):
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        for data, targets in train_loader:
            data = data.to(device)
            # Ensure targets are not empty and have the correct shape
            if not all(target.numel() > 0 for target in targets):  # Check if all target tensors have elements
                print("Warning: Empty target batch encountered. Skipping this batch.")
                continue  # Skip this batch if any target is empty

            targets = [target.to(device) for target in targets]

            optimizer.zero_grad()
            outputs = model(data)
            losses = [criterion(output, target) for output, target in zip(outputs, targets)]
            loss = sum(losses)

            print("[EPOCH]: %i, [LOSS]: %.6f" % (epoch, loss.item()))
            display.clear_output(wait=True)

            loss.backward()
            optimizer.step()

# Example usage:
# Assuming train_loader and val_loader are defined and provide a list of targets for each data point

learning_rate = 0.01
epochs = 10

model = OptimizedCNN(num_classes_list).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training
train_model(model, train_loader, optimizer, epochs)

# Evaluation
evaluate_model(model, validation_loader)
evaluate_model(model, test_loader, is_test=True)

In [ ]:
'''def evaluate_model(model, dataloader, is_test=False):
  #Evaluation

  # Set model to evaluation mode
  model.eval()

  with torch.no_grad():
    correct = 0
    loss = 0

    criterion = torch.nn.CrossEntropyLoss(reduction='none')

    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)

        loss += torch.sum(criterion(outputs, target)).item()

        pred = outputs.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    loss /= len(dataloader.dataset)
    accuracy = 100. * correct / len(dataloader.dataset)
    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        "Test" if is_test else "Validation",
        loss, correct, len(dataloader.dataset),
        accuracy))
  # Set model back to training mode
  model.train()

###### Your code starts here. ######

def train_model(M):
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        for input, label in train_loader:
            input, label = input.to(device), label.to(device)
            pred_label = M(input)
            optimizer.zero_grad()
            loss = criterion(pred_label, label)
            print("[EPOCH]: %i, [LOSS or MSE]: %.6f" % (epoch, loss.item()))
            display.clear_output(wait=True)
            loss.backward()
            optimizer.step()


###### Your code end here. ######
learning_rate = 0.01
epochs = 30'''

In [ ]:
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Using Adam optimizer with learning rate 0.001

train_model(model)



In [ ]:
evaluate_model(model, validation_loader, is_test=True)

In [ ]:
for a_type in _ :
  count = a_type[a_type]
  if count < 300:
    count_to_change = (0.18 * count) / 3
  else:
    count_to_change = (0.05 * count) / 3
  change_im_df = traindf[a_type].sample(frac=count_to_change)
  for image in change_im_df :
    sheared_im = image.clone().shear('Black', 20, 30) # should print out and check if this is too awkward..
    pixeliated_im = image.clone().resize(300, 226) # should print out and check if this is too grainy..
    darker_im = ImageEnhance.Brightness(image.clone()).enhance() # should print out and check if this is too dark..
    altered_df = pd.DataFrame({a_type:[sheared_im, pixeliated_im, darker_im]})
    traindf.append(altered_df, ignore_index=True) # might be more complicated than this ...
